<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-6/blob/main/ML_Zoomcamp_Week_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 10/10/2022**

In [57]:
# Import all necessary libraries that we will use in this project
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [58]:
# Download the dataset for the project
data = data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'

!wget $data

--2022-10-14 09:30:55--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182489 (178K) [text/plain]
Saving to: ‘CreditScoring.csv.1’

CreditScoring.csv.1 100%[===================>] 178.21K  --.-KB/s    in 0.009s  

2022-10-14 09:30:55 (18.9 MB/s) - ‘CreditScoring.csv.1’ saved [182489/182489]



In [59]:
# Print the column names and the first 9 records of the dataset
!head CreditScoring.csv

"Status","Seniority","Home","Time","Age","Marital","Records","Job","Expenses","Income","Assets","Debt","Amount","Price"
1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
1,0,1,36,26,1,1,1,46,107,0,0,310,910
1,1,2,60,36,2,1,1,75,214,3500,0,650,1645
1,29,2,60,44,2,1,1,75,125,10000,0,1600,1800
1,9,5,12,27,1,1,1,35,80,0,0,200,1093
1,0,2,60,32,2,1,3,90,107,15000,0,1200,1957


In [60]:
# Print the data now using Pandas
df = pd.read_csv(data)

In [61]:
# Print the first 5 rows of our daraset
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [62]:
# Preparing the data:

# Use the column status we want to translate the numbers into strings
# with text is easier to understand, what this status means

In [63]:
# Lower case all the column names of the dataset
df.columns = df.columns.str.lower()
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [64]:
# Check the column status and how many values we have
df.status.value_counts()
# we can see we have a variable zero

1    3200
2    1254
0       1
Name: status, dtype: int64

In [65]:
# Convert the column status to string using the map function
# this function map converts to a dictionary every value from 
# the original data frame to some new value
df.status.map({1: 'ok', 2: 'default', 0: 'unk'}) # unk means unknown

0            ok
1            ok
2       default
3            ok
4            ok
         ...   
4450    default
4451         ok
4452    default
4453         ok
4454         ok
Name: status, Length: 4455, dtype: object

In [66]:
# Check our dataframe after used the map function
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [67]:
# Now dealing with values: home, marital, records and job
home_values = {
    1: 'rent',
    2: 'owner',
    3: 'private',
    4: 'ignore',
    5: 'parents',
    6: 'other',
    0: 'unk'
}

df.home = df.home.map(home_values)

marital_values = {
    1: 'single',
    2: 'married',
    3: 'widow',
    4: 'separated',
    5: 'divorced',
    0: 'unk'
}

df.marital = df.marital.map(marital_values)

records_values = {
    1: 'no',
    2: 'yes',
    0: 'unk'
}

df.records = df.records.map(records_values)

job_values = {
    1: 'fixed',
    2: 'partime',
    3: 'freelance',
    4: 'others',
    0: 'unk'
}

df.job = df.job.map(job_values)

In [68]:
# Check our data frame after used the map function
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,rent,60,30,married,no,freelance,73,129,0,0,800,846
1,1,17,rent,60,58,widow,no,fixed,48,131,0,0,1000,1658
2,2,10,owner,36,46,married,yes,freelance,90,200,3000,0,2000,2985
3,1,0,rent,60,24,single,no,fixed,63,182,2500,0,900,1325
4,1,0,rent,36,26,single,no,fixed,46,107,0,0,310,910


In [69]:
# Use the method describe that describes the dataset like all the statistics
df.describe()

,status,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.000000,4455.000000,4455.000000,4455.000000,4455.000000,4.455000e+03,4.455000e+03,4.455000e+03,4455.000000,4455.000000
mean,1.281257,7.987205,46.441751,37.077666,55.568799,7.633170e+05,1.060341e+06,4.043820e+05,1039.021773,1462.875645
std,0.450162,8.173444,14.655225,10.984856,19.515878,8.703625e+06,1.021757e+07,6.344253e+06,474.543007,628.089913
min,0.000000,0.000000,6.000000,18.000000,35.000000,0.000000e+00,0.000000e+00,0.000000e+00,100.000000,105.000000
25%,1.000000,2.000000,36.000000,28.000000,35.000000,8.000000e+01,0.000000e+00,0.000000e+00,700.000000,1117.500000
50%,1.000000,5.000000,48.000000,36.000000,51.000000,1.200000e+02,3.500000e+03,0.000000e+00,1000.000000,1400.000000
75%,2.000000,12.000000,60.000000,45.000000,72.000000,1.660000e+02,6.000000e+03,0.000000e+00,1300.000000,1692.000000
max,2.000000,48.000000,72.000000,68.000000,180.000000,1.000000e+08,1.000000e+08,1.000000e+08,5000.000000,11140.000000


In [70]:
# Use the method describe with the random function
df.describe().round()
# below we can see income, assets and debt have large numbers as max values
# this max values means the value is missing

,status,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0
mean,1.0,8.0,46.0,37.0,56.0,763317.0,1060341.0,404382.0,1039.0,1463.0
std,0.0,8.0,15.0,11.0,20.0,8703625.0,10217569.0,6344253.0,475.0,628.0
min,0.0,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,1.0,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,1.0,5.0,48.0,36.0,51.0,120.0,3500.0,0.0,1000.0,1400.0
75%,2.0,12.0,60.0,45.0,72.0,166.0,6000.0,0.0,1300.0,1692.0
max,2.0,48.0,72.0,68.0,180.0,99999999.0,99999999.0,99999999.0,5000.0,11140.0


In [71]:
# Using another way similar to map function to replace values
df.income.replace(to_replace=99999999, value=np.nan)

0       129.0
1       131.0
2       200.0
3       182.0
4       107.0
        ...  
4450     92.0
4451     75.0
4452     90.0
4453    140.0
4454    140.0
Name: income, Length: 4455, dtype: float64

In [72]:
# Create a for loop to update the max value with NaN
for c in ['income', 'assets','debt']:

  # After access the column c we want to replace the value with NaN
  df [c] = df[c].replace(to_replace=99999999, value=np.nan)

In [73]:
# Check the max value is gone from the dataframe
df.describe().round() 
# we can see below we don't have the max value anymore

,status,seniority,time,age,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4421.0,4408.0,4437.0,4455.0,4455.0
mean,1.0,8.0,46.0,37.0,56.0,131.0,5403.0,343.0,1039.0,1463.0
std,0.0,8.0,15.0,11.0,20.0,86.0,11573.0,1246.0,475.0,628.0
min,0.0,0.0,6.0,18.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,1.0,2.0,36.0,28.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,1.0,5.0,48.0,36.0,51.0,120.0,3000.0,0.0,1000.0,1400.0
75%,2.0,12.0,60.0,45.0,72.0,165.0,6000.0,0.0,1300.0,1692.0
max,2.0,48.0,72.0,68.0,180.0,959.0,300000.0,30000.0,5000.0,11140.0


In [74]:
# Look at the status variable. We have zeros meaning we have missing values
df.status.value_counts()
# below the status zero means unk and we want to remove it

1    3200
2    1254
0       1
Name: status, dtype: int64

In [75]:
# Filter all records with status, not zero, from the data frame
df[df.status != 0]

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,rent,60,30,married,no,freelance,73,129.0,0.0,0.0,800,846
1,1,17,rent,60,58,widow,no,fixed,48,131.0,0.0,0.0,1000,1658
2,2,10,owner,36,46,married,yes,freelance,90,200.0,3000.0,0.0,2000,2985
3,1,0,rent,60,24,single,no,fixed,63,182.0,2500.0,0.0,900,1325
4,1,0,rent,36,26,single,no,fixed,46,107.0,0.0,0.0,310,910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4450,2,1,rent,60,39,married,no,fixed,69,92.0,0.0,0.0,900,1020
4451,1,22,owner,60,46,married,no,fixed,60,75.0,3000.0,600.0,950,1263
4452,2,0,owner,24,37,married,no,partime,60,90.0,3500.0,0.0,500,963
4453,1,0,rent,48,23,single,no,freelance,49,140.0,0.0,0.0,550,550


In [76]:
# Reset the index of the data frame
df = df[df.status != 0].reset_index()
df

,index,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,0,1,9,rent,60,30,married,no,freelance,73,129.0,0.0,0.0,800,846
1,1,1,17,rent,60,58,widow,no,fixed,48,131.0,0.0,0.0,1000,1658
2,2,2,10,owner,36,46,married,yes,freelance,90,200.0,3000.0,0.0,2000,2985
3,3,1,0,rent,60,24,single,no,fixed,63,182.0,2500.0,0.0,900,1325
4,4,1,0,rent,36,26,single,no,fixed,46,107.0,0.0,0.0,310,910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4449,4450,2,1,rent,60,39,married,no,fixed,69,92.0,0.0,0.0,900,1020
4450,4451,1,22,owner,60,46,married,no,fixed,60,75.0,3000.0,600.0,950,1263
4451,4452,2,0,owner,24,37,married,no,partime,60,90.0,3500.0,0.0,500,963
4452,4453,1,0,rent,48,23,single,no,freelance,49,140.0,0.0,0.0,550,550


In [77]:
# Doing the train, validation and test split:

# Import library sklearn the package train test split
from sklearn.model_selection import train_test_split

# Split the data into full train and test dataframe
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

# Split the data into train and validation dataframe
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=11)

In [78]:
# Reset the index of the train, validation and test dataframes
df_train = df_train.reset_index(drop=True)
df_val   = df_val.reset_index(drop=True)
df_test  = df_test.reset_index(drop=True)

In [79]:
# Turn the status to a number 1 = true and 0 = false and store in y_train variable
y_train = (df_train.status == 2).astype('int').values
y_train
# Do the same to validation and test dataframes
y_val  = (df_val.status == 2).astype('int').values
y_test = (df_test.status == 2).astype('int').values

In [80]:
# Remove the default variables from train, validation and test dataframes
del df_train['status']
del df_val['status']
del df_test['status']

In [81]:
df_train

,index,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,951,10,owner,36,36,married,no,freelance,75,0.0,10000.0,0.0,1000,1400
1,688,6,parents,48,32,single,yes,fixed,35,85.0,0.0,0.0,1100,1330
2,2233,1,parents,48,40,married,no,fixed,75,121.0,0.0,0.0,1320,1600
3,3304,1,parents,48,23,single,no,partime,35,72.0,0.0,0.0,1078,1079
4,2271,5,owner,36,46,married,no,freelance,60,100.0,4000.0,0.0,1100,1897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,2382,18,private,36,45,married,no,fixed,45,220.0,20000.0,0.0,800,1600
2668,1784,7,private,60,29,married,no,fixed,60,51.0,3500.0,500.0,1000,1290
2669,808,1,parents,24,19,single,no,fixed,35,28.0,0.0,0.0,400,600
2670,1857,15,owner,48,43,married,no,freelance,60,100.0,18000.0,0.0,2500,2976


In [82]:
# Applying Decision Trees:

# Create asses risk function
def assess_risk(client):

  # Check the customer has a record
  if client['records'] == 'yes':

    # Check the customer job is part-time
    if client['job'] == 'parttime':
      return 'default'
    else:
      return 'ok'
  else:
    # Check the customer have more than 6k in assets
    if client['assets'] > 6000:
      return 'ok'
    else:
      return 'default'

In [83]:
# Convert the first record from the train dataframe into a dictionary
xi = df_train.iloc[0].to_dict()
xi

{'index': 951,
 'seniority': 10,
 'home': 'owner',
 'time': 36,
 'age': 36,
 'marital': 'married',
 'records': 'no',
 'job': 'freelance',
 'expenses': 75,
 'income': 0.0,
 'assets': 10000.0,
 'debt': 0.0,
 'amount': 1000,
 'price': 1400}

In [84]:
# Use the assess risk function with our customer
assess_risk(xi)

'ok'

In [85]:
# Now we can train using the rules created above
# so we can learn these rules from the data using the decision tree algorithm
# and for that we are using the sklearn

# Import sklearn with the package Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

In [86]:
# Use dictionary vectorizer because we have categorical variables
from sklearn.feature_extraction import DictVectorizer

In [87]:
# Import sklearn with the ROC AUC score
from sklearn.metrics import roc_auc_score

In [88]:
# Import the library warnings to ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [89]:
# Turn our train dataframe into a list of dictionaries
# and then turn the list of dictionaries into the feature matrix
# and then after we will train the model
# Use the fillna function to fill the NaN with zeros
train_dicts = df_train.fillna(0).to_dict(orient='records')

In [90]:
# Create a new instance of the DictVectorizer class without sparse
dv = DictVectorizer(sparse=False)

# Use the function transform with our DictVectorizer
X_train = dv.fit_transform(train_dicts)

In [91]:
# Look and check at the feature names
dv.get_feature_names()

['age',
 'amount',
 'assets',
 'debt',
 'expenses',
 'home=ignore',
 'home=other',
 'home=owner',
 'home=parents',
 'home=private',
 'home=rent',
 'home=unk',
 'income',
 'index',
 'job=fixed',
 'job=freelance',
 'job=others',
 'job=partime',
 'job=unk',
 'marital=divorced',
 'marital=married',
 'marital=separated',
 'marital=single',
 'marital=unk',
 'marital=widow',
 'price',
 'records=no',
 'records=yes',
 'seniority',
 'time']

In [92]:
# Train our decision tree
dt = DecisionTreeClassifier()

# For training the model we use the fit method
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [93]:
# Test our model:

# Use the validation dataframe to transform to a list of dictionaries
val_dicts = df_val.fillna(0).to_dict(orient='records')

# Use the function transform with our DictVectorizer
X_val = dv.transform(val_dicts)

In [94]:
# Apply our model on X validation and use the first column
y_pred = dt.predict_proba(X_val)[:, 1]

In [95]:
# Compute the ROC AUC score
roc_auc_score(y_val, y_pred)

0.6499327940713472

In [96]:
# Let's see the ROC AUC score for our train dataframe:

# Apply our model on X train and use the first column
y_pred = dt.predict_proba(X_train)[:, 1]

# Compute the ROC AUC score
roc_auc_score(y_train, y_pred)
# the score below is 100%, and it's excellent but 
# the model learned from the data, and we don't want this

1.0

In [97]:
# Retrain the tree so the model doesn't learn from the data
# and for that we control the depth of the tree

# Train our decision tree with max depth of 3 levels
dt = DecisionTreeClassifier(max_depth=3)

# For training the model we use the fit method
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [98]:
# Apply our model on X train and use the first column
y_pred = dt.predict_proba(X_train)[:, 1]

# Compute the ROC AUC score
auc = roc_auc_score(y_train, y_pred)

# Print the result of the ROC AUC score for the train dataframe
print('Train:', auc)

# Apply our model on X train and use the first column
y_pred = dt.predict_proba(X_val)[:, 1]

# Compute the ROC AUC score
auc = roc_auc_score(y_val, y_pred)

# Print the result of the ROC AUC score for the validation dataframe
print('Validation:', auc)

# below, the performance of our model on the validation dataframe is significantly better
# it's 73% compared to 65%

Train: 0.7761016984958594
Validation: 0.7389079944782155


In [99]:
# Use sklearn tree that helps to visualise trees
from sklearn.tree import export_text

In [100]:
# Print the train decision tree
print(export_text(dt))

|--- feature_27 <= 0.50
|   |--- feature_17 <= 0.50
|   |   |--- feature_12 <= 74.50
|   |   |   |--- class: 0
|   |   |--- feature_12 >  74.50
|   |   |   |--- class: 0
|   |--- feature_17 >  0.50
|   |   |--- feature_2 <= 8750.00
|   |   |   |--- class: 1
|   |   |--- feature_2 >  8750.00
|   |   |   |--- class: 0
|--- feature_27 >  0.50
|   |--- feature_28 <= 6.50
|   |   |--- feature_1 <= 862.50
|   |   |   |--- class: 0
|   |   |--- feature_1 >  862.50
|   |   |   |--- class: 1
|   |--- feature_28 >  6.50
|   |   |--- feature_12 <= 103.50
|   |   |   |--- class: 1
|   |   |--- feature_12 >  103.50
|   |   |   |--- class: 0



In [101]:
# Print the train decision tree with feature names
print(export_text(dt, feature_names=dv.get_feature_names()))

|--- records=yes <= 0.50
|   |--- job=partime <= 0.50
|   |   |--- income <= 74.50
|   |   |   |--- class: 0
|   |   |--- income >  74.50
|   |   |   |--- class: 0
|   |--- job=partime >  0.50
|   |   |--- assets <= 8750.00
|   |   |   |--- class: 1
|   |   |--- assets >  8750.00
|   |   |   |--- class: 0
|--- records=yes >  0.50
|   |--- seniority <= 6.50
|   |   |--- amount <= 862.50
|   |   |   |--- class: 0
|   |   |--- amount >  862.50
|   |   |   |--- class: 1
|   |--- seniority >  6.50
|   |   |--- income <= 103.50
|   |   |   |--- class: 1
|   |   |--- income >  103.50
|   |   |   |--- class: 0



In [102]:
# Decision Tree Learning Algorithm:

# Use simple data for decision tree learning algorithm
data = [
    [8000, 'default'],
    [2000, 'default'],
    [   0, 'default'],
    [5000, 'ok'],
    [5000, 'ok'],
    [4000, 'ok'],
    [9000, 'ok'],
    [3000, 'default'],
]

In [103]:
# Convert the data to a dataframe example
df_example = pd.DataFrame(data)
df_example

,0,1
0,8000,default
1,2000,default
2,0,default
3,5000,ok
4,5000,ok
5,4000,ok
6,9000,ok
7,3000,default


In [104]:
# Define the column names of the example dataframe
df_example = pd.DataFrame(data, columns=['assets','status'])
df_example

,assets,status
0,8000,default
1,2000,default
2,0,default
3,5000,ok
4,5000,ok
5,4000,ok
6,9000,ok
7,3000,default


In [105]:
# Use the numerical column assets to train our decision tree
# using the assets column 

# Sort the dataframe example by assets
df_example.sort_values('assets')

# below we can see the assets values and we want to come up
# with a rule assets greater than threshold and the possibilities
# are split the dataframe with left part T = 2000 T = 3000 and then cut the dataframe
# and then the right part with T = 4000 T = 5000 and T = 8000
# note: we don't need assets with zero because is the first node
# and we don't need the last node with 9000
# so the potential thresholds are: 2000, 3000, 4000, 5000 and 8000

,assets,status
2,0,default
1,2000,default
7,3000,default
5,4000,ok
3,5000,ok
4,5000,ok
0,8000,default
6,9000,ok


In [106]:
# Create a list with the potential thresholds
Ts = [0, 2000, 3000, 4000, 5000, 8000]

In [107]:
# Import library display from Python
from IPython.display import display

In [108]:
# now we want to cut our dataframe example in the left and right side
# and see which split is the best one

# Create a for loop to use our list of thresholds
for T in Ts:

  # Print the threshold value
  print(T)

  # Split the data if the assets are equal or less greater than the threshold
  df_left = df_example[df_example.assets <= T]

  # Split the data if the assets are equal or greater than the threshold
  df_right = df_example[df_example.assets > T]

  # Print data from the left and right of the dataframe example
  display(df_left)
  display(df_right)

  # Print blank line
  print()

0


,assets,status
2,0,default


,assets,status
0,8000,default
1,2000,default
3,5000,ok
4,5000,ok
5,4000,ok
6,9000,ok
7,3000,default



2000


,assets,status
1,2000,default
2,0,default


...

,assets,status
1,2000,default
2,0,default
3,5000,ok
4,5000,ok
5,4000,ok
7,3000,default


,assets,status
0,8000,default
6,9000,ok



8000


,assets,status
0,8000,default
1,2000,default
2,0,default
3,5000,ok
4,5000,ok
5,4000,ok
7,3000,default


,assets,status
6,9000,ok


In [109]:
# Now we have many splits, and we want to know which split
# is the best for our evaluating because we have different evaluation criteria
# or split evaluation criteria 

# Let's have an example using the threshold with 4000
T = 4000

# Split the data if the assets are equal or less greater than the threshold
df_left = df_example[df_example.assets <= T]

# Split the data if the assets are equal or greater than the threshold
df_right = df_example[df_example.assets > T]

# Print data from the left and right of the dataframe example
display(df_left)

# Use the function value counts with status and normalize set to true to return ratios
# instead of the absolute number
print(df_left.status.value_counts(normalize=True))

# Use the function value counts with status and normalize to the right side now
display(df_right)
print(df_right.status.value_counts(normalize=True))

,assets,status
1,2000,default
2,0,default
5,4000,ok
7,3000,default


default    0.75
ok         0.25
Name: status, dtype: float64


,assets,status
0,8000,default
3,5000,ok
4,5000,ok
6,9000,ok


ok         0.75
default    0.25
Name: status, dtype: float64


In [110]:
# To understand how good our predictions are here, we can look at
# misclassification rate and is when we predict everyone as default
# how many errors we make, so what is the fraction of errors here 
# we can see above start from the top we have 3 default and 1 ok
# so we predict everyone as default and we'll make one error out of 4
# because we have 4 assets and our misclassification rate is 25%
# and this will be our left. 
# On the right we have 3 ok and one default so then we predict everyone is okay
# and we have missed classification rate of 25% 
# doing this we can evaluate the quality of our split 
# so for T = 4000 we get 25 error rate misclassification rate
# on the left 25% and on the right 25%, and then we can take an average
# and the average is 25%, and we can say if we use this T = 4000 as the split, then
# the average misclassification rate is 25% 
# we don't have to take coverage so that we can take a weighted average, for example
# we have one example on the left and eight examples on the right, and it makes more
# it makes sense to take a weighted average so that we will use the one example of more weight
# but for simplicity, we will use the usual average
# in real life, the way these algorithms work is they use a weighted average

In [111]:
# Now let's do the same but now inside of for loop:

# Create a for loop to use with the list of thresholds
for T in Ts:

  # Print the threshold value
  print(T)

  # Split the data if the assets are equal or less greater than the threshold
  df_left = df_example[df_example.assets <= T]

  # Split the data if the assets are equal or greater than the threshold
  df_right = df_example[df_example.assets > T]

  # Print data from the left and right of the dataframe example
  display(df_left)

  # Use the function value counts with status and normalize set to true to return ratios
  # instead of the absolute number
  print(df_left.status.value_counts(normalize=True))

  # Use the function value counts with status and normalize to the right side now
  display(df_right)
  print(df_right.status.value_counts(normalize=True))

  # Print blank line
  print()

0


,assets,status
2,0,default


default    1.0
Name: status, dtype: float64


,assets,status
0,8000,default
1,2000,default
3,5000,ok
4,5000,ok
5,4000,ok
6,9000,ok
7,3000,default


ok         0.571429
default    0.428571
Name: status, dtype: float64

2000


...

default    0.5
ok         0.5
Name: status, dtype: float64

8000


,assets,status
0,8000,default
1,2000,default
2,0,default
3,5000,ok
4,5000,ok
5,4000,ok
7,3000,default


default    0.571429
ok         0.428571
Name: status, dtype: float64


,assets,status
6,9000,ok


ok    1.0
Name: status, dtype: float64



In [112]:
# Use simple data for decision tree learning algorithm

# Add a second column feature called debt to the dataset
data = [
    [8000, 3000, 'default'],
    [2000, 1000, 'default'],
    [   0, 1000, 'default'],
    [5000, 1000, 'ok'],
    [5000, 1000, 'ok'],
    [4000, 1000, 'ok'],
    [9000,  500, 'ok'],
    [3000, 2000, 'default'],
]

# Define the column names of the example dataframe
df_example = pd.DataFrame(data, columns=['assets', 'debt', 'status'])
df_example

,assets,debt,status
0,8000,3000,default
1,2000,1000,default
2,0,1000,default
3,5000,1000,ok
4,5000,1000,ok
5,4000,1000,ok
6,9000,500,ok
7,3000,2000,default


In [113]:
# Sort the values by debt with the dataframe example
df_example.sort_values('debt')

,assets,debt,status
6,9000,500,ok
1,2000,1000,default
2,0,1000,default
3,5000,1000,ok
4,5000,1000,ok
5,4000,1000,ok
7,3000,2000,default
0,8000,3000,default


In [114]:
# Look at the table above. We can see we can split the dataset by:
# 500, 1000 and 2000 (we look at the values with status ok)

# Create a dictionary with values collected from assets and debt
thresholds = {
    'assets': [0, 2000, 3000, 4000, 5000, 8000],
    'debt': [500, 1000, 2000]
}

In [115]:
# Iterate over each of the customers for feature and threshold items
for feature, Ts in thresholds.items():

  # Print the feature name of the dataframe
  print('#######################')
  print(feature)
  
  # Iterate over the list of thresholds
  for T in Ts:

    # Now instead of doing with assets we do for all features
  
    # Print the threshold value
    print(T)

    # Split the data if the assets are equal or less greater than the threshold
    # df_left = df_example[df_example.assets <= T]
    df_left = df_example[df_example[feature] <= T]

    # Split the data if the assets are equal or greater than the threshold
    # df_right = df_example[df_example.assets > T]
    df_right = df_example[df_example[feature] > T]

    # Print data from the left and right of the dataframe example
    display(df_left)

    # Use the function value counts with status and normalize set to true to return ratios
    # instead of the absolute number
    print(df_left.status.value_counts(normalize=True))

    # Use the function value counts with status and normalize to the right side now
    display(df_right)
    print(df_right.status.value_counts(normalize=True))

    # Print blank line
    print()

# Print a separate line for better visualization of the data
print('#######################')

#######################
assets
0


,assets,debt,status
2,0,1000,default


default    1.0
Name: status, dtype: float64


,assets,debt,status
0,8000,3000,default
1,2000,1000,default
3,5000,1000,ok
4,5000,1000,ok
5,4000,1000,ok
6,9000,500,ok
7,3000,2000,default


ok         0.571429
default    0.428571
Name: status, dtype: float64

2000


...

default    1.0
Name: status, dtype: float64

2000


,assets,debt,status
1,2000,1000,default
2,0,1000,default
3,5000,1000,ok
4,5000,1000,ok
5,4000,1000,ok
6,9000,500,ok
7,3000,2000,default


ok         0.571429
default    0.428571
Name: status, dtype: float64


,assets,debt,status
0,8000,3000,default


default    1.0
Name: status, dtype: float64

#######################


In [ ]:
# Decision Tree Learning Algorithm:
# . Find the best split
# . Stop IF Max_Depth is reached
# . IF LEFT is sufficiently large
#        AND NOT pure:
#.         REPEAT FOR LEFT
# . IF RIGHT is sufficiently large
#        AND NOT pure:
#.         REPEAT FOR RIGHT

In [ ]:
# Decision Tree Parameter Tunning:
# . Selecting max_depth
# . Selecting min_samples_leaf

In [ ]:
# Iterate over different values of max depth
# None means no restriction and it should grow tree as deep as possible
# as many layers as possible 
for d in [1, 2, 3, 4, 5, 6, 10, 15, 20 , None]:

  # Train a decision tree by adjusting the max depth parameter
  dt = DecisionTreeClassifier()

  # For training the model we use the fit method
  dt.fit(X_train, y_train)

  # Apply our model on X validation and use the first column
  y_pred = dt.predict_proba(X_val)[:, 1]